In [1]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib

# Load JSON data into DataFrame
with open("data.json", "r") as file:
    data = json.load(file)

df = pd.DataFrame(data)

# Get all fields from the DataFrame
all_fields = df.columns.tolist()

# Function to train and save a model for each target variable
def train_models(df, all_fields):
    models = {}
    for target in all_fields:
        if target == "Sale Code":
            continue  # Skip Sale Code as it's not a suitable target
        
        # Use all other fields as features
        features = [field for field in all_fields if field != target and field != "Sale Code"]
        
        # Encode categorical features
        categorical_features = [field for field in features if df[field].dtype == 'object']
        numerical_features = [field for field in features if df[field].dtype != 'object']
        
        preprocessor = ColumnTransformer(
            transformers=[
                ('cat', OneHotEncoder(), categorical_features),
                ('num', StandardScaler(), numerical_features)
            ],
            remainder='passthrough'
        )
        
        X = df[features]
        y = df[target]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Check if the target is categorical or numerical
        if df[target].dtype == 'object':
            model = Pipeline(steps=[
                ('preprocessor', preprocessor),
                ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
            ])
        else:
            model = Pipeline(steps=[
                ('preprocessor', preprocessor),
                ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
            ])
        
        model.fit(X_train, y_train)
        
        # Save the model
        joblib.dump(model, "final_price_model.pkl")
        models[target] = model
    return models

# Train models for all potential target variables
models = train_models(df, all_fields)

# Function to predict any target field based on given inputs
def predict(inputs, target_field):
    if target_field not in models:
        raise ValueError(f"No model trained for predicting {target_field}")
    
    model = models[target_field]
    
    # Ensure all expected input fields are provided
    features = [field for field in all_fields if field != target_field and field != "Sale Code"]
    missing_fields = set(features) - set(inputs.keys())
    if missing_fields:
        raise ValueError(f"Input is missing fields: {missing_fields}")
    
    prediction = model.predict(pd.DataFrame([inputs]))
    return prediction[0]

# Example usage
example_input = {
    "First Name": "Apple",
    "Last Name": "Electronics",
    "Buyer ID": "APP2741",
    "Brand": "Audi",
    "Model": "A4",
    "Manufacture Country": "GE",
    "Sale Country": "US"
}

# Predict the final price
final_price_prediction = predict(example_input, "Final Price")
print(f"Predicted Final Price: {final_price_prediction}")

Predicted Final Price: 36540.0


# Testing for every field

In [2]:
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
import joblib

# Load data
with open("data.json", "r") as file:
    data = json.load(file)

# Convert to DataFrame
df = pd.DataFrame(data)

# Identify categorical features dynamically
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

# Encode categorical features
label_encoders = {}
for feature in categorical_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le

# Function to train and save a model for each target variable
def train_and_evaluate_models(df, all_fields, categorical_features):
    models = {}
    performance = {}
    for target in all_fields:
        if target == "Sale Code":
            continue  # Skip Sale Code as it's not a suitable target

        # Use all other fields as features
        features = [field for field in all_fields if field != target and field != "Sale Code"]

        X = df[features]
        y = df[target]

        # Split the data into training and test sets randomly
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Check if the target is categorical or numerical
        if df[target].dtype == 'object':
            model = RandomForestClassifier(n_estimators=100, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            score = accuracy_score(y_test, y_pred)
            performance[target] = score
            print(score)
            if score > 0.95:
                joblib.dump(model, f'{target}_model.pkl')
                models[target] = model
        else:
            print(target)
            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            performance[target] = mse
            #print(mse)
            if mse < 500:
                joblib.dump(model, f'{target}_model.pkl')
                models[target] = model

    return models, performance

# Get all fields from the DataFrame
all_fields = df.columns.tolist()

# Train and evaluate models for all potential target variables
models, performance = train_and_evaluate_models(df, all_fields, categorical_features)

# Function to predict any target field based on given inputs
def predict(inputs, target_field):
    if target_field not in models:
        raise ValueError(f"No model trained for predicting {target_field}")

    model = models[target_field]

    # Convert new data to DataFrame
    new_df = pd.DataFrame([inputs])

    # Encode new data
    for feature in categorical_features:
        if feature in inputs:
            new_df[feature] = label_encoders[feature].transform(new_df[feature])

    prediction = model.predict(new_df)
    return prediction[0]

# Example usage to predict "Final Price"
example_input = {
    "First Name": "Apple",
    "Last Name": "Electronics",
    "Buyer ID": "APP2741",
    "Brand": "Audi",
    "Model": "A4",
    "Manufacture Country": "GE",
    "Sale Country": "US"
}


# Predict the final price if the model exists
if "Final Price" in models:
    final_price_prediction = predict(example_input, "Final Price")
    print(f"Predicted Final Price: {final_price_prediction}")
else:
    print("No model available for predicting Final Price.")

# Output performance of models
print("Model performance:")
for target, score in performance.items():
    print(f"{target}: {score}")


First Name
Last Name
Buyer ID
Brand
Model
Manufacture Country
Sale Country
Final Price
Predicted Final Price: 36540.0
Model performance:
First Name: 0.0
Last Name: 0.0
Buyer ID: 0.0
Brand: 0.0
Model: 0.0
Manufacture Country: 0.1567212965562381
Sale Country: 8.250000000000009e-06
Final Price: 329.79721999999765


In [3]:
# Example usage to predict "Final Price"
example_input = {
    "First Name": "Google",
    "Last Name": "Data",
    "Buyer ID": "GOO4226",
    "Brand": "Mercedes",
    "Model": "C-Class",
    "Manufacture Country": "GE",
    "Sale Country": "GE",
        
}


# Predict the final price if the model exists
if "Final Price" in models:
    final_price_prediction = predict(example_input, "Final Price")
    print(f"Predicted Final Price: {final_price_prediction}")
else:
    print("No model available for predicting Final Price.")

Predicted Final Price: 38500.0


In [4]:
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
import joblib

# Load data
with open("data.json", "r") as file:
    data = json.load(file)

# Convert to DataFrame
df = pd.DataFrame(data)

# Identify categorical features dynamically
original_categorical_features = df.select_dtypes(include=['object']).columns.tolist()

# Encode categorical features
label_encoders = {}
for feature in original_categorical_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le

# Function to train and save a model for each target variable
def train_and_evaluate_models(df, all_fields, original_categorical_features):
    models = {}
    performance = {}
    for target in all_fields:
        if target == "Sale Code":
            continue  # Skip Sale Code as it's not a suitable target

        # Use all other fields as features
        features = [field for field in all_fields if field != target and field != "Sale Code"]

        X = df[features]
        y = df[target]

        # Split the data into training and test sets randomly
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

        # Check if the target is originally categorical or numerical
        if target in original_categorical_features:
            model = RandomForestClassifier(n_estimators=100, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            score = accuracy_score(y_test, y_pred)
            performance[target] = score
            if score > 0.95:
                joblib.dump(model, f'{target}_model.pkl')
                models[target] = model
        else:
            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            performance[target] = mse
            print(f"{target} MSE: {mse}")
            if mse < 500:
                joblib.dump(model, f'{target}_model.pkl')
                models[target] = model

    return models, performance

# Get all fields from the DataFrame
all_fields = df.columns.tolist()

# Train and evaluate models for all potential target variables
models, performance = train_and_evaluate_models(df, all_fields, original_categorical_features)

# Function to predict any target field based on given inputs
def predict(inputs, target_field):
    if target_field not in models:
        raise ValueError(f"No model trained for predicting {target_field}")

    model = models[target_field]

    # Convert new data to DataFrame
    new_df = pd.DataFrame([inputs])

    # Encode new data
    for feature in original_categorical_features:
        if feature in inputs:
            new_df[feature] = label_encoders[feature].transform(new_df[feature])

    prediction = model.predict(new_df)
    
    # Decode prediction if the target is categorical
    if target_field in original_categorical_features:
        prediction = label_encoders[target_field].inverse_transform([prediction[0]])[0]
    
    return prediction

# Example usage to predict "Last Name"
example_input = {
    "First Name": "Apple",
    "Last Name": "Electronics",
    "Buyer ID": "APP2741",
    "Brand": "Audi",
    "Model": "A4",
    "Manufacture Country": "GE",
    "Sale Country": "US"
}

# Predict the last name if the model exists
if "Final Price" in models:
    last_name_prediction = predict(example_input, "Final Price")
    print(f"Predicted Last Name: {last_name_prediction}")
else:
    print("No model available for predicting Last Name.")

# Output performance of models
print("Model performance:")
for target, score in performance.items():
    print(f"{target}: {score}")


Final Price MSE: 329.79721999999765
Predicted Last Name: [36540.]
Model performance:
First Name: 1.0
Last Name: 1.0
Buyer ID: 1.0
Brand: 1.0
Model: 1.0
Manufacture Country: 0.86
Sale Country: 1.0
Final Price: 329.79721999999765


# Predictability of each target variable with others 

In [5]:
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
import joblib

# Load data
with open("data.json", "r") as file:
    data = json.load(file)

# Convert to DataFrame
df = pd.DataFrame(data)

# Identify categorical features dynamically
original_categorical_features = df.select_dtypes(include=['object']).columns.tolist()

# Encode categorical features
label_encoders = {}
for feature in original_categorical_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le

# Function to evaluate the predictability of a target field
def evaluate_predictability(df, target_field, original_categorical_features):
    performance = {}
    features = [field for field in df.columns if field != target_field and field != "Sale Code"]
    
    # Iteratively add features and evaluate performance
    for i in range(1, len(features) + 1):
        current_features = features[:i]
        X = df[current_features]
        y = df[target_field]

        # Split the data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

        # Check if the target is originally categorical or numerical
        if target_field in original_categorical_features:
            model = RandomForestClassifier(n_estimators=100, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            score = accuracy_score(y_test, y_pred)
            performance[f"Using first {i} features"] = score
        else:
            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            performance[f"Using first {i} features"] = mse

        print(f"Using first {i} features: {performance[f'Using first {i} features']}")

    return performance

# Evaluate predictability for all fields
all_fields = df.columns.tolist()
for target_field in all_fields:
    if target_field == "Sale Code":
        continue  # Skip Sale Code as it's not a suitable target

    print(f"\nEvaluating predictability for target field '{target_field}':")
    predictability_performance = evaluate_predictability(df, target_field, original_categorical_features)

    # Output the predictability performance
    print(f"\nPredictability performance for target field '{target_field}':")
    for features_set, score in predictability_performance.items():
        print(f"{features_set}: {score}")



Evaluating predictability for target field 'First Name':
Using first 1 features: 0.917
Using first 2 features: 1.0
Using first 3 features: 1.0
Using first 4 features: 1.0
Using first 5 features: 1.0
Using first 6 features: 1.0
Using first 7 features: 1.0

Predictability performance for target field 'First Name':
Using first 1 features: 0.917
Using first 2 features: 1.0
Using first 3 features: 1.0
Using first 4 features: 1.0
Using first 5 features: 1.0
Using first 6 features: 1.0
Using first 7 features: 1.0

Evaluating predictability for target field 'Last Name':
Using first 1 features: 1.0
Using first 2 features: 1.0
Using first 3 features: 1.0
Using first 4 features: 1.0
Using first 5 features: 1.0
Using first 6 features: 1.0
Using first 7 features: 1.0

Predictability performance for target field 'Last Name':
Using first 1 features: 1.0
Using first 2 features: 1.0
Using first 3 features: 1.0
Using first 4 features: 1.0
Using first 5 features: 1.0
Using first 6 features: 1.0
Using fi

# Model for only First and last name

In [14]:
import json
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# Load data
with open("data.json", "r") as file:
    data = json.load(file)

# Convert to DataFrame
df = pd.DataFrame(data)

# Identify categorical features dynamically
original_categorical_features = df.select_dtypes(include=['object']).columns.tolist()

# Encode categorical features
label_encoders = {}
for feature in original_categorical_features:
    le = LabelEncoder()
    df[feature] = le.fit_transform(df[feature])
    label_encoders[feature] = le

# Function to train and save a model for each target variable
def train_and_evaluate_model_for_last_name(df, original_categorical_features):
    target = "Last Name"
    features = ["First Name"]  # Use only "First Name" as the feature
    
    X = df[features]
    y = df[target]

    # Split the data into training and test sets randomly
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

    # Train the model for "Last Name"
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    performance = score
    print(f"Last Name Accuracy: {score}")
    if score > 0.95:
        joblib.dump(model, 'using_first_name_for_last_name_model.pkl')
        joblib.dump(label_encoders, 'label_encoders.pkl')
    return performance

# Train and evaluate the model for "Last Name"
models = {}
performance_last_name = train_and_evaluate_model_for_last_name(df, original_categorical_features)

# Function to predict any target field based on given inputs
def predict(inputs, target_field):
    if target_field not in models:
        raise ValueError(f"No model trained for predicting {target_field}")

    model = models[target_field]

    # Convert new data to DataFrame
    new_df = pd.DataFrame([inputs])

    # Encode new data
    for feature in original_categorical_features:
        if feature in inputs:
            new_df[feature] = label_encoders[feature].transform(new_df[feature])

    prediction = model.predict(new_df)
    
    # Decode prediction if the target is categorical
    if target_field in original_categorical_features:
        prediction = label_encoders[target_field].inverse_transform(prediction)
    
    return prediction[0]

# Example usage to predict "Last Name"
example_input = {
    "First Name": "Michael"
}

# Predict the last name if the model exists
if "Last Name" in models:
    last_name_prediction = predict(example_input, "Last Name")
    print(f"Predicted Last Name: {last_name_prediction}")
else:
    print("No model available for predicting Last Name.")

# Output performance of models
print("Model performance for Last Name:")
print(f"Using only 'First Name': {performance_last_name}")


Last Name Accuracy: 1.0
No model available for predicting Last Name.
Model performance for Last Name:
Using only 'First Name': 1.0


# Making some form of ui

In [ ]:
import tkinter as tk
from tkinter import StringVar
import joblib
import pandas as pd

# Load the trained model and label encoders
model = joblib.load('using_first_name_for_last_name_model.pkl')
label_encoders = joblib.load('label_encoders.pkl')

# Function to predict "Last Name" based on "First Name"
def predict_last_name(first_name):
    if first_name:
        inputs = {"First Name": first_name}
        new_df = pd.DataFrame([inputs])
        new_df["First Name"] = label_encoders["First Name"].transform(new_df["First Name"])

        prediction = model.predict(new_df)
        prediction = label_encoders["Last Name"].inverse_transform(prediction)
        return prediction[0]
    return ""

# Update predictions in real-time
def update_prediction(*args):
    first_name = first_name_var.get()
    last_name = predict_last_name(first_name)
    last_name_var.set(last_name)

# Create the main window
root = tk.Tk()
root.title("Real-Time Prediction")

# Define StringVars
first_name_var = StringVar()
last_name_var = StringVar()

# Bind the update function to the first name input
first_name_var.trace_add("write", update_prediction)

# Create and place labels and entry widgets
tk.Label(root, text="First Name").grid(row=0, column=0)
tk.Entry(root, textvariable=first_name_var).grid(row=0, column=1)
tk.Label(root, text="Last Name").grid(row=1, column=0)
tk.Entry(root, textvariable=last_name_var, state='readonly').grid(row=1, column=1)

# Run the main event loop
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 225, in _encode
    return _map_to_integer(values, uniques)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 165, in _map_to_integer
    return np.array([table[v] for v in values])
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 165, in <listcomp>
    return np.array([table[v] for v in values])
                     ~~~~~^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 159, in __missing__
    raise KeyError(key)
KeyError: 'M'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 225, in _encode
    return _map_to_integer(values, uniques)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 165, in _map_to_integer
    return np.array([table[v] for v in values])
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 165, in <listcomp>
    return np.array([table[v] for v in values])
                     ~~~~~^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 159, in __missing__
    raise KeyError(key)
KeyError: 'Micros'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, 

In [ ]:
import tkinter as tk
from tkinter import StringVar
import joblib
import pandas as pd

# Load the trained model and label encoders
models = {
    "Last Name": joblib.load('using_first_name_for_last_name_model.pkl'),
    # Add other models as needed
}
label_encoders = joblib.load('label_encoders.pkl')

# Function to predict fields based on the current inputs
def predict_field(inputs, target_field):
    if target_field in models:
        model = models[target_field]
        new_df = pd.DataFrame([inputs])
        
        # Encode input features
        for feature in inputs.keys():
            if feature in label_encoders and inputs[feature]:  # Only encode non-empty values
                new_df[feature] = label_encoders[feature].transform([inputs[feature]])
        
        # Make prediction
        prediction = model.predict(new_df)
        
        # Decode the prediction if the target field is categorical
        if target_field in label_encoders:
            prediction = label_encoders[target_field].inverse_transform(prediction)
        
        return prediction[0]
    return ""

# Update predictions in real-time
def update_predictions(*args):
    current_inputs = {
        "First Name": first_name_var.get(),
        "Last Name": last_name_var.get(),
        "Buyer ID": buyer_id_var.get(),
        "Brand": brand_var.get(),
        "Model": model_var.get(),
        "Manufacture Country": manufacture_country_var.get(),
        "Sale Country": sale_country_var.get()
    }

    # Only consider non-empty inputs for prediction
    non_empty_inputs = {k: v for k, v in current_inputs.items() if v}
    print("Current Inputs:", non_empty_inputs)  # Debugging statement

    if "First Name" in non_empty_inputs:
        predicted_last_name = predict_field(non_empty_inputs, "Last Name")
        print("Predicted Last Name:", predicted_last_name)  # Debugging statement
        if not last_name_var.get():  # Only update if the field is empty
            last_name_var.set(predicted_last_name)

    # Add other prediction updates as needed

# Clear the last name when first name changes
def first_name_changed(*args):
    last_name_var.set("")
    update_predictions()

# Create the main window
root = tk.Tk()
root.title("Real-Time Prediction Form")

# Define StringVars
first_name_var = StringVar()
last_name_var = StringVar()
buyer_id_var = StringVar()
brand_var = StringVar()
model_var = StringVar()
manufacture_country_var = StringVar()
sale_country_var = StringVar()

# Bind the update function to the input variables
first_name_var.trace_add("write", first_name_changed)
buyer_id_var.trace_add("write", update_predictions)
brand_var.trace_add("write", update_predictions)
model_var.trace_add("write", update_predictions)
manufacture_country_var.trace_add("write", update_predictions)
sale_country_var.trace_add("write", update_predictions)

# Create and place labels and entry widgets
tk.Label(root, text="First Name").grid(row=0, column=0)
tk.Entry(root, textvariable=first_name_var).grid(row=0, column=1)
tk.Label(root, text="Last Name").grid(row=1, column=0)
tk.Entry(root, textvariable=last_name_var).grid(row=1, column=1)
tk.Label(root, text="Buyer ID").grid(row=2, column=0)
tk.Entry(root, textvariable=buyer_id_var).grid(row=2, column=1)
tk.Label(root, text="Brand").grid(row=3, column=0)
tk.Entry(root, textvariable=brand_var).grid(row=3, column=1)
tk.Label(root, text="Model").grid(row=4, column=0)
tk.Entry(root, textvariable=model_var).grid(row=4, column=1)
tk.Label(root, text="Manufacture Country").grid(row=5, column=0)
tk.Entry(root, textvariable=manufacture_country_var).grid(row=5, column=1)
tk.Label(root, text="Sale Country").grid(row=6, column=0)
tk.Entry(root, textvariable=sale_country_var).grid(row=6, column=1)

# Run the main event loop
root.mainloop()


Current Inputs: {'First Name': 'M'}
Current Inputs: {'First Name': 'Mi'}


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 225, in _encode
    return _map_to_integer(values, uniques)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 165, in _map_to_integer
    return np.array([table[v] for v in values])
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 165, in <listcomp>
    return np.array([table[v] for v in values])
                     ~~~~~^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 159, in __missing__
    raise KeyError(key)
KeyError: 'M'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __

Current Inputs: {'First Name': 'Mic'}


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 225, in _encode
    return _map_to_integer(values, uniques)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 165, in _map_to_integer
    return np.array([table[v] for v in values])
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 165, in <listcomp>
    return np.array([table[v] for v in values])
                     ~~~~~^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 159, in __missing__
    raise KeyError(key)
KeyError: 'Mic'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in 

Current Inputs: {'First Name': 'Micr'}


...

Current Inputs: {'First Name': 'Micro'}
Current Inputs: {'First Name': 'Micros'}


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 225, in _encode
    return _map_to_integer(values, uniques)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 165, in _map_to_integer
    return np.array([table[v] for v in values])
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 165, in <listcomp>
    return np.array([table[v] for v in values])
                     ~~~~~^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 159, in __missing__
    raise KeyError(key)
KeyError: 'Micro'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, i

Current Inputs: {'First Name': 'Microso'}
Current Inputs: {'First Name': 'Microsof'}


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 225, in _encode
    return _map_to_integer(values, uniques)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 165, in _map_to_integer
    return np.array([table[v] for v in values])
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 165, in <listcomp>
    return np.array([table[v] for v in values])
                     ~~~~~^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/utils/_encode.py", line 159, in __missing__
    raise KeyError(key)
KeyError: 'Microso'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948,

Current Inputs: {'First Name': 'Microsoft'}
Predicted Last Name: Software


In [ ]:
import tkinter as tk
from tkinter import StringVar
import joblib
import pandas as pd

# Load the trained models and label encoders
models = {
    "Last Name": joblib.load('using_first_name_for_last_name_model.pkl'),
    "Final Price": joblib.load('final_price_model.pkl')
}
label_encoders = joblib.load('label_encoders.pkl')

# Features used during training
features_for_final_price = ["First Name", "Last Name", "Buyer ID", "Brand", "Model", "Manufacture Country", "Sale Country"]

# Function to handle unseen labels gracefully
def transform_with_encoder(encoder, values):
    unique_labels = encoder.classes_
    transformed_values = []
    for value in values:
        if value in unique_labels:
            transformed_values.append(encoder.transform([value])[0])
        else:
            transformed_values.append(-1)  # Using -1 for unseen labels
    return transformed_values

# Function to predict fields based on the current inputs
def predict_field(inputs, target_field, feature_names):
    if target_field in models:
        model = models[target_field]
        new_df = pd.DataFrame([inputs])
        
        # Ensure all feature columns are present
        for feature in feature_names:
            if feature not in new_df:
                new_df[feature] = ""
        
        # Print the DataFrame before encoding
        print(f"DataFrame before encoding for {target_field}:")
        print(new_df)
        
        # Encode input features
        for feature in new_df.columns:
            if feature in label_encoders:
                new_df[feature] = transform_with_encoder(label_encoders[feature], new_df[feature])
        
        # Print the DataFrame after encoding
        print(f"DataFrame after encoding for {target_field}:")
        print(new_df)
        
        # Make prediction
        prediction = model.predict(new_df)
        
        # Print the prediction
        print(f"Prediction for {target_field}: {prediction}")
        
        # Decode the prediction if the target field is categorical
        if target_field in label_encoders:
            prediction = label_encoders[target_field].inverse_transform(prediction)
        
        return prediction[0]
    return ""

# Update predictions in real-time
def update_predictions(*args):
    current_inputs = {
        "First Name": first_name_var.get(),
        "Last Name": last_name_var.get(),
        "Buyer ID": buyer_id_var.get(),
        "Brand": brand_var.get(),
        "Model": model_var.get(),
        "Manufacture Country": manufacture_country_var.get(),
        "Sale Country": sale_country_var.get(),
        "Final Price": final_price_var.get()
    }

    # Only consider non-empty inputs for prediction
    non_empty_inputs = {k: v for k, v in current_inputs.items() if v}
    print("Current Inputs:", non_empty_inputs)  # Debugging statement

    # Predict Last Name if First Name is available
    if "First Name" in non_empty_inputs:
        predicted_last_name = predict_field(non_empty_inputs, "Last Name", ["First Name"])
        print("Predicted Last Name:", predicted_last_name)  # Debugging statement
        if not last_name_var.get():  # Only update if the field is empty
            last_name_var.set(predicted_last_name)

    # Predict Final Price if all necessary inputs are available
    if all(field in non_empty_inputs for field in features_for_final_price):
        print("Predicting Final Price with inputs:", non_empty_inputs)  # Additional debug statement
        predicted_final_price = predict_field(non_empty_inputs, "Final Price", features_for_final_price)
        print("Predicted Final Price:", predicted_final_price)  # Debugging statement
        final_price_var.set(predicted_final_price)

# Clear the last name when first name changes
def first_name_changed(*args):
    last_name_var.set("")
    update_predictions()

# Create the main window
root = tk.Tk()
root.title("Real-Time Prediction Form")

# Define StringVars
first_name_var = StringVar()
last_name_var = StringVar()
buyer_id_var = StringVar()
brand_var = StringVar()
model_var = StringVar()
manufacture_country_var = StringVar()
sale_country_var = StringVar()
final_price_var = StringVar()

# Bind the update function to the input variables
first_name_var.trace_add("write", first_name_changed)
last_name_var.trace_add("write", update_predictions)
buyer_id_var.trace_add("write", update_predictions)
brand_var.trace_add("write", update_predictions)
model_var.trace_add("write", update_predictions)
manufacture_country_var.trace_add("write", update_predictions)
sale_country_var.trace_add("write", update_predictions)
final_price_var.trace_add("write", update_predictions)

# Create and place labels and entry widgets
tk.Label(root, text="First Name").grid(row=0, column=0)
tk.Entry(root, textvariable=first_name_var).grid(row=0, column=1)
tk.Label(root, text="Last Name").grid(row=1, column=0)
tk.Entry(root, textvariable=last_name_var).grid(row=1, column=1)
tk.Label(root, text="Buyer ID").grid(row=2, column=0)
tk.Entry(root, textvariable=buyer_id_var).grid(row=2, column=1)
tk.Label(root, text="Brand").grid(row=3, column=0)
tk.Entry(root, textvariable=brand_var).grid(row=3, column=1)
tk.Label(root, text="Model").grid(row=4, column=0)
tk.Entry(root, textvariable=model_var).grid(row=4, column=1)
tk.Label(root, text="Manufacture Country").grid(row=5, column=0)
tk.Entry(root, textvariable=manufacture_country_var).grid(row=5, column=1)
tk.Label(root, text="Sale Country").grid(row=6, column=0)
tk.Entry(root, textvariable=sale_country_var).grid(row=6, column=1)
tk.Label(root, text="Final Price").grid(row=7, column=0)
tk.Entry(root, textvariable=final_price_var).grid(row=7, column=1)  # Input box for final price

# Run the main event loop
root.mainloop()


Current Inputs: {'First Name': 'M'}
DataFrame before encoding for Last Name:
  First Name
0          M
DataFrame after encoding for Last Name:
   First Name
0          -1
Prediction for Last Name: [3]
Predicted Last Name: Electronics
Current Inputs: {'First Name': 'M', 'Last Name': 'Electronics'}
DataFrame before encoding for Last Name:
  First Name    Last Name
0          M  Electronics
DataFrame after encoding for Last Name:
   First Name  Last Name
0          -1          3
Current Inputs: {'First Name': 'Mi'}
DataFrame before encoding for Last Name:
  First Name
0         Mi
DataFrame after encoding for Last Name:
   First Name
0          -1
Prediction for Last Name: [3]
Predicted Last Name: Electronics
Current Inputs: {'First Name': 'Mi', 'Last Name': 'Electronics'}
DataFrame before encoding for Last Name:
  First Name    Last Name
0         Mi  Electronics
DataFrame after encoding for Last Name:
   First Name  Last Name
0          -1          3


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/ipykernel_39334/2027498353.py", line 98, in first_name_changed
    update_predictions()
  File "/var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/ipykernel_39334/2027498353.py", line 83, in update_predictions
    predicted_last_name = predict_field(non_empty_inputs, "Last Name", ["First Name"])
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/ipykernel_39334/2027498353.py", line 52, in predict_field
    prediction = model.predict(new_df)
                 ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py", line 904, in predict
    proba = self.predict_proba(X)
           

Current Inputs: {'First Name': 'Mic'}
DataFrame before encoding for Last Name:
  First Name
0        Mic
DataFrame after encoding for Last Name:
   First Name
0          -1
Prediction for Last Name: [3]
Predicted Last Name: Electronics
Current Inputs: {'First Name': 'Mic', 'Last Name': 'Electronics'}
DataFrame before encoding for Last Name:
  First Name    Last Name
0        Mic  Electronics
DataFrame after encoding for Last Name:
   First Name  Last Name
0          -1          3
Current Inputs: {'First Name': 'Mico'}
DataFrame before encoding for Last Name:
  First Name
0       Mico
DataFrame after encoding for Last Name:
   First Name
0          -1
Prediction for Last Name: [3]
Predicted Last Name: Electronics
Current Inputs: {'First Name': 'Mico', 'Last Name': 'Electronics'}
DataFrame before encoding for Last Name:
  First Name    Last Name
0       Mico  Electronics
DataFrame after encoding for Last Name:
   First Name  Last Name
0          -1          3


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/ipykernel_39334/2027498353.py", line 98, in first_name_changed
    update_predictions()
  File "/var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/ipykernel_39334/2027498353.py", line 83, in update_predictions
    predicted_last_name = predict_field(non_empty_inputs, "Last Name", ["First Name"])
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/ipykernel_39334/2027498353.py", line 52, in predict_field
    prediction = model.predict(new_df)
                 ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py", line 904, in predict
    proba = self.predict_proba(X)
           

Current Inputs: {'First Name': 'Mic'}
DataFrame before encoding for Last Name:
  First Name
0        Mic
DataFrame after encoding for Last Name:
   First Name
0          -1
Prediction for Last Name: [3]
Predicted Last Name: Electronics
Current Inputs: {'First Name': 'Mic', 'Last Name': 'Electronics'}
DataFrame before encoding for Last Name:
  First Name    Last Name
0        Mic  Electronics
DataFrame after encoding for Last Name:
   First Name  Last Name
0          -1          3
Current Inputs: {'First Name': 'Micr'}
DataFrame before encoding for Last Name:
  First Name
0       Micr
DataFrame after encoding for Last Name:
   First Name
0          -1
Prediction for Last Name: [3]
Predicted Last Name: Electronics
Current Inputs: {'First Name': 'Micr', 'Last Name': 'Electronics'}
DataFrame before encoding for Last Name:
  First Name    Last Name
0       Micr  Electronics
DataFrame after encoding for Last Name:
   First Name  Last Name
0          -1          3
Current Inputs: {'First Name

...

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/ipykernel_39334/2027498353.py", line 83, in update_predictions
    predicted_last_name = predict_field(non_empty_inputs, "Last Name", ["First Name"])
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/ipykernel_39334/2027498353.py", line 52, in predict_field
    prediction = model.predict(new_df)
                 ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py", line 904, in predict
    proba = self.predict_proba(X)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py", line 946, in predict_proba
    X

Current Inputs: {'First Name': 'Microsoft', 'Last Name': 'Software', 'Buyer ID': 'MIC8821', 'Brand': 'Audi', 'Model': 'A8', 'Manufacture Country': 'G'}
DataFrame before encoding for Last Name:
  First Name Last Name Buyer ID Brand Model Manufacture Country
0  Microsoft  Software  MIC8821  Audi    A8                   G
DataFrame after encoding for Last Name:
   First Name  Last Name  Buyer ID  Brand  Model  Manufacture Country
0           7          7         7      0      8                   -1
Current Inputs: {'First Name': 'Microsoft', 'Last Name': 'Software', 'Buyer ID': 'MIC8821', 'Brand': 'Audi', 'Model': 'A8', 'Manufacture Country': 'GE'}
DataFrame before encoding for Last Name:
  First Name Last Name Buyer ID Brand Model Manufacture Country
0  Microsoft  Software  MIC8821  Audi    A8                  GE
DataFrame after encoding for Last Name:
   First Name  Last Name  Buyer ID  Brand  Model  Manufacture Country
0           7          7         7      0      8                   

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/ipykernel_39334/2027498353.py", line 83, in update_predictions
    predicted_last_name = predict_field(non_empty_inputs, "Last Name", ["First Name"])
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/ipykernel_39334/2027498353.py", line 52, in predict_field
    prediction = model.predict(new_df)
                 ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py", line 904, in predict
    proba = self.predict_proba(X)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py", line 946, in predict_proba
    X

Current Inputs: {'First Name': 'Microsoft', 'Last Name': 'Software', 'Buyer ID': 'MIC8821', 'Brand': 'Audi', 'Model': 'A8', 'Manufacture Country': 'GE', 'Sale Country': 'G'}
DataFrame before encoding for Last Name:
  First Name Last Name Buyer ID Brand Model Manufacture Country Sale Country
0  Microsoft  Software  MIC8821  Audi    A8                  GE            G
DataFrame after encoding for Last Name:
   First Name  Last Name  Buyer ID  Brand  Model  Manufacture Country  \
0           7          7         7      0      8                    0   

   Sale Country  
0            -1  
Current Inputs: {'First Name': 'Microsoft', 'Last Name': 'Software', 'Buyer ID': 'MIC8821', 'Brand': 'Audi', 'Model': 'A8', 'Manufacture Country': 'GE', 'Sale Country': 'GE'}
DataFrame before encoding for Last Name:
  First Name Last Name Buyer ID Brand Model Manufacture Country Sale Country
0  Microsoft  Software  MIC8821  Audi    A8                  GE           GE
DataFrame after encoding for Last Name

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ayush/anaconda3/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/ipykernel_39334/2027498353.py", line 83, in update_predictions
    predicted_last_name = predict_field(non_empty_inputs, "Last Name", ["First Name"])
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/j8/s3w557v576v0dtnghwcjfxm40000gn/T/ipykernel_39334/2027498353.py", line 52, in predict_field
    prediction = model.predict(new_df)
                 ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py", line 904, in predict
    proba = self.predict_proba(X)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ayush/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py", line 946, in predict_proba
    X